# Cryosphere model Comparison tool (CmCt) --- IMBIE

The CmCt IMBIE tool compares user-uploaded ice sheet modeled mass change to reconciled mass change observations from the Ice-sheet Mass Balance Intercomparison Exercise (IMBIE). The IMBIE data is provided as ice-sheet-integrated time series of mass change. In the future, IMBIE will partition mass change into drainage basins, as well as into total mass balance, surface mass balance, and dynamic mass balance and this tool is designed to be able to process those. The CmCt uses a drainage basin mask to partition modeled mass change into the separate basins and sums mass changes across all basins (`Masked_Total` in the output results). The CmCt also sums modeled mass change for the entire gridded model, without applying any basin masking ('Unmasked_Total` in the output results). Note that these two sums may be different, if the gridded model file contains grid cells that are outside of the IMBIE drainage basin mask.

## Input data requirements

The input ice sheet model needs to be provided as a netCDF file. The user may upload a single input file that includes multiple years; future enhancements to this tool will allow users to upload model ensembles.

There are several requirements for the comparison:

### `Lithk` variable

The CmCt Grace Mascon tool expects the uploaded model to contain thickness data (the `lithk` variable) for the comparison.

### Rectangular grid

At time of writing, models *must* be defined on a rectangular X-Y grid in the ISMIP6 standard projected polar-stereographic space. (Note, NOT a lat-lon grid!) The ISMIP6 standard projection is defined [here](https://theghub.org/groups/ismip6/wiki). 

### Date range

The gravimetry data spans 04/2002 to 12/2023. The user can select start and end dates within this span as part of the setup for the tool.

## Instructions to use the tool

- Select a model: GIS or AIS.
- Upload .nc file: If you selected GIS model, upload GIS model related .nc file otherwise upload AIS model related .nc file
- Choose dates: The gravimetry data spans 04/2002 to 12/2023. So, select start date and end date within the span.
- Select mass balance: From the dropdown select 'Cumulative mass balance' or 'Cumulative dynamics mass balance anomaly'
- Run process to compare the input data against IMBIE observations. You can download results in csv file.

In [ ]:

import os, sys
import cftime
import datetime
import xarray as xr
import ipywidgets as widgets
from IPython.display import display, clear_output

# Add the directory containing 'cmct' to the Python path
cmct_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
cur_dir=os.path.dirname(os.path.realpath(" "))
sys.path.insert(0, cmct_dir)
from cmct.imbie import *

# Use a writable directory inside your home path
upload_directory = cmct_dir+'/uploaded_file'

output=widgets.Output()
upload_output = widgets.Output()
process_output = widgets.Output()
result_output = widgets.Output()

# Ensure the directory exists
os.makedirs(upload_directory, exist_ok=True)

global nc_filename

models=['GIS','AIS']
# Global variable to store the uploaded file path
icesheet_widget = widgets.Dropdown(
    options=models,
    value='GIS',
    description='Ice Sheet:'
)

#File Upload Widget
upload_widget = widgets.FileUpload(
    accept='.nc',
    multiple=False
)
upload_widget.disabled = (icesheet_widget.value == 'None')

#Start Date Picker
start_date_widget = widgets.DatePicker(
    description='Start Date:',
    disabled=True
)

#End Date Picker
end_date_widget = widgets.DatePicker(
    description='End Date:',
    disabled=True
)

#Mass Balance Dropdown
mass_balance_widget = widgets.Dropdown(
    options=[
        ('Cumulative mass balance (Gt)', 'total'),
        ('Cumulative dynamics mass balance anomaly (Gt)', 'dynamic')
    ],
    value='total',
    description='Mass Balance:',
    disabled=True,
    style={'description_width': 'initial'}
)

#Run Process Button
run_button = widgets.Button(
    description="Run Process",
    disabled=True
)

#Function to update file paths AFTER a file is uploaded
def update_file_paths():
    global projection, shape_filename, obs_filename, obs_east_filename, obs_west_filename, obs_peninsula_filename

    icesheet = icesheet_widget.value  # Get the currently selected Ice Sheet
    with output:
        clear_output(wait=True)

    if icesheet == 'GIS':
        projection = 'EPSG:3413'
        shape_filename = cmct_dir + '/data/IMBIE/Greenland_Basins_PS_v1.4.2/Greenland_Basins_PS_v1.4.2.shp'
        obs_filename = cmct_dir + '/data/IMBIE/imbie_greenland_2021_Gt.csv'
        obs_east_filename, obs_west_filename, obs_peninsula_filename = None, None, None

    elif icesheet == 'AIS':
        projection = 'EPSG:3031'
        shape_filename = cmct_dir + '/data/IMBIE/ANT_Basins_IMBIE2_v1.6/ANT_Basins_IMBIE2_v1.6.shp'
        obs_filename = cmct_dir + '/data/IMBIE/imbie_antarctica_2021_Gt.csv'
        obs_east_filename, obs_west_filename, obs_peninsula_filename = None, None, None

#Function to handle Ice Sheet selection
def on_icesheet_change(change):
    
    # Enable upload button after GIS/AIS selection
    upload_widget.disabled = (change['new'] == 'None') 
    update_file_paths()


#Function to handle file upload
def on_upload_change_orig(change):
    upload_output.clear_output(wait=True)
    with upload_output:
        clear_output(wait=True)
        global uploaded_nc_file
        global nc_filename
        uploaded_nc_file = None  # Reset previous file

        if not upload_widget.value:
            with upload_output:
                print("No file detected in FileUpload widget!")
            return

        for file in os.listdir(upload_directory):
            file_path = os.path.join(upload_directory, file)
            try:
                os.remove(file_path)
            except Exception as e:
                with upload_output:
                    print(f"Error deleting file {file_path}: {e}")

        try:
            # Extract file content correctly
            uploaded_file = list(upload_widget.value.values())[0]  # Extract file dict
            file_name = uploaded_file['metadata']['name']  # Get file name
            file_content = uploaded_file['content']  # Get file binary content
            file_path = os.path.join(upload_directory, file_name)  # Target path

            # Write file to the specified directory
            with open(file_path, "wb") as f:
                f.write(file_content)

            print('File saved successfully')
            uploaded_nc_file = file_path  # Store path globally
            nc_filename = uploaded_nc_file

            update_file_paths()

            mod_ds = xr.open_dataset(uploaded_nc_file, use_cftime=True)
            time_var = mod_ds['time']
            min_time = time_var.values.min()
            max_time = time_var.values.max()
            print(f"Model data time range is from {min_time} to {max_time}.Choose dates accordingly")

            start_date_widget.disabled = False

        except Exception as e:
            with upload_output:
                clear_output(wait=True)
                print(f"Error during file upload: {e}")

def on_upload_change(change):
    upload_output.clear_output(wait=True)
    try:
        for file in os.listdir(upload_directory):
            file_path = os.path.join(upload_directory, file)
            try:
                os.remove(file_path)
            except Exception as e:
                with upload_output:
                    print(f"Error deleting file {file_path}: {e}")

        uploaded_file = list(upload_widget.value.values())[0]
        file_name = uploaded_file['metadata']['name']
        file_content = uploaded_file['content']
        file_path = os.path.join(upload_directory, file_name)

        with open(file_path, "wb") as f:
            f.write(file_content)

        global uploaded_nc_file, nc_filename
        uploaded_nc_file = file_path
        nc_filename = uploaded_nc_file

        update_file_paths()

        mod_ds = xr.open_dataset(uploaded_nc_file, use_cftime=True)
        time_var = mod_ds['time']
        min_time = time_var.values.min()
        max_time = time_var.values.max()

        with upload_output:
            print("File saved successfully")
            print(f"Model data time range is from {min_time} to {max_time}. Choose dates accordingly")

        start_date_widget.disabled = False

    except Exception as e:
        with upload_output:
            clear_output(wait=True)
            print(f"Error during file upload: {e}")


# Function to enable end date after selecting start date
def on_start_date_change(change):
    with output:
        clear_output(wait=True)
    if start_date_widget.value:
        end_date_widget.disabled = False  

# Function to enable mass balance dropdown after selecting end date
def on_end_date_change(change):
    with output:
        clear_output(wait=True)
    if end_date_widget.value:  # Check if a date is selected
        mass_balance_widget.disabled = False
        if mass_balance_widget.value:
            run_button.disabled = False

# Function to enable run button after selecting mass balance
def on_mass_balance_change(change):
    with output:
        clear_output(wait=True)
    if mass_balance_widget.value:
        run_button.disabled = False  


# Attach event listeners to widgets
icesheet_widget.observe(on_icesheet_change, names='value')
upload_widget.observe(on_upload_change, names='value')
start_date_widget.observe(on_start_date_change, names='value')
end_date_widget.observe(on_end_date_change, names='value')
mass_balance_widget.observe(on_mass_balance_change, names='value')

display(icesheet_widget)
display(upload_widget)
display(upload_output)
display(start_date_widget)
display(end_date_widget)
display(mass_balance_widget)
display(run_button)


In [ ]:
def check_files():
    # Check if observation file exists
    if not os.path.exists(obs_filename):
        raise FileNotFoundError(f"Observation file not found: {obs_filename}")

    # Check if model file exists    
    if not os.path.exists(nc_filename):
        raise FileNotFoundError(f"Model file not found: {nc_filename}")
    
    icesheet = icesheet_widget.value
    
    if icesheet == 'AIS':
        if (obs_east_filename and os.path.exists(obs_east_filename)) and \
               (obs_west_filename and os.path.exists(obs_west_filename)) and \
               (obs_peninsula_filename and os.path.exists(obs_peninsula_filename)):
            # Check if regional observation files exist 
            if not os.path.exists(obs_east_filename):
                raise FileNotFoundError(f"Observation file not found: {obs_east_filename}")
            if not os.path.exists(obs_west_filename):
                raise FileNotFoundError(f"Observation file not found: {obs_west_filename}")
            if not os.path.exists(obs_peninsula_filename):
                raise FileNotFoundError(f"Observation file not found: {obs_peninsula_filename}")

In [ ]:
import hublib
import hublib.ui as ui
display(process_output)
# Function to run the process when all selections are made
def on_run_process_clicked(b):
    process_output.clear_output(wait=True)
    with process_output:
        clear_output(wait=True)
        run_button.disabled = True
        print("Running the process...")
        

        # Convert start/end comparison times to fractional year
        try:
            check_files()
            # Open model file
            mod_ds = xr.open_dataset(uploaded_nc_file, use_cftime=True)
            time_var = mod_ds['time']
            calendar_type = time_var.to_index().calendar
            start_date_dt = start_date_widget.value
            end_date_dt = end_date_widget.value

            # Adjust day to avoid errors in non-Gregorian calendars
            start_date_cftime = cftime.datetime(start_date_dt.year, start_date_dt.month, min(start_date_dt.day, 30), calendar=calendar_type)
            end_date_cftime = cftime.datetime(end_date_dt.year, end_date_dt.month, min(end_date_dt.day, 30), calendar=calendar_type)

            start_date_fract = start_date_cftime.year + (start_date_cftime.dayofyr - 1) / 365
            end_date_fract = end_date_cftime.year + (end_date_cftime.dayofyr - 1) / 365

            # Mapping of user-friendly options to actual column names
            mass_balance_column_mapping = {
                'total': 'Cumulative mass balance (Gt)',
                'dynamic': 'Cumulative dynamics mass balance anomaly (Gt)'
            }

            # Get the correct column name based on user selection
            mass_balance_type = mass_balance_widget.value
            mass_balance_column = mass_balance_column_mapping[mass_balance_type]

            # Read IMBIE mass change from start to end date
            IMBIE_mass_change = process_imbie_data(obs_filename, start_date_fract, end_date_fract, mass_balance_column)
            
            # Calculate time-varying modeled mass change
            model_mass_change = process_model_data(
                mod_ds, time_var, IMBIE_mass_change, start_date_cftime, end_date_cftime, start_date_fract, end_date_fract, 918, projection, shape_filename, icesheet_widget.value
            )

            # Calculate IMBIE-model mass change residuals
            imbie_model_residuals = calculate_model_imbie_residuals(
                start_date_fract, end_date_fract, icesheet_widget.value, model_mass_change, IMBIE_mass_change, mass_balance_type, None, None, None
            )

            # Extract the base name of the uploaded .nc file (without extension)
            nc_base_filename = os.path.basename(uploaded_nc_file).replace('.nc', '')


            # Create the CSV filename
            csv_filename = os.path.join(cur_dir, f"{nc_base_filename}.csv")
            
            download_button=hublib.ui.Download(os.path.relpath(csv_filename,cur_dir),
                label = 'Download CSV', style='success', icon='fa-arrow-circle-down')
            clear_output(wait=True)
            display(download_button)
            
            print("Preparing download... Click the button only after results are fully displayed")

            write_and_display_mass_change_comparison_all_dates(
                icesheet_widget.value, model_mass_change, imbie_model_residuals, mass_balance_type, start_date_fract, end_date_fract, csv_filename
            )

            print(f"Process completed successfully! Output saved to: {csv_filename}")

            
        except Exception as e:
                print(f"Error: {e}") 
        finally:
            run_button.disabled = False

run_button.on_click(on_run_process_clicked)